# LLM Cookbook with Intel Gaudi

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/llama3_cookbook_gaudi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meta developed and released the Meta [Llama 3](https://ai.meta.com/blog/meta-llama-3/) family of large language models (LLMs), a collection of pretrained and instruction tuned generative text models in 8 and 70B sizes. The Llama 3 instruction tuned models are optimized for dialogue use cases and outperform many of the available open source chat models on common industry benchmarks.

In this notebook, we will demonstrate how to use Llama3 with LlamaIndex. 

We use Llama-3-8B-Instruct for the demonstration through Intel Gaudi.

## Installation and Setup

In [ ]:
!pip install llama-parse
!pip install python-dotenv==1.0.0
!pip install llama_index
!pip install llama-index-llms-gaudi
!pip install llama-index-embeddings-gaudi
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-readers-wikipedia
!pip install wikipedia
!pip install InstructorEmbedding==1.0.1
!pip install sentence-transformers
!pip install --upgrade-strategy eager optimum[habana]
!pip install optimum-habana==1.14.1
!pip install huggingface-hub==0.23.2

INFO: pip is looking at multiple versions of optimum-habana to determine which version is compatible with other requirements. This could take a while.
  Using cached optimum_habana-1.14.0-py3-none-any.whl.metadata (24 kB)
  Using cached optimum_habana-1.13.2-py3-none-any.whl.metadata (23 kB)
  Using cached optimum_habana-1.13.1-py3-none-any.whl.metadata (23 kB)
  Using cached optimum_habana-1.13.0-py3-none-any.whl.metadata (23 kB)
  Using cached optimum_habana-1.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached optimum_habana-1.12.0-py3-none-any.whl.metadata (21 kB)
  Using cached optimum_habana-1.11.1-py3-none-any.whl.metadata (18 kB)
INFO: pip is still looking at multiple versions of optimum-habana to determine which version is compatible with other requirements. This could take a while.
  Using cached optimum_habana-1.11.0-py3-none-any.whl.metadata (18 kB)
  Using cached optimum_habana-1.10.4-py3-none-any.whl.metadata (16 kB)
  Using cached optimum_habana-1.10.2-py3-none-any.whl

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import argparse
import os, sys, logging

from llama_index.readers.wikipedia import WikipediaReader
from llama_index.llms.gaudi import GaudiLLM
from llama_index.embeddings.gaudi import GaudiEmbedding
from llama_index.core.prompts import PromptTemplate

from llama_index.core import (
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    Settings,
    StorageContext,
)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [ ]:
def setup_parser(parser):
    # Arguments management
    parser.add_argument(
        "--device",
        "-d",
        type=str,
        choices=["hpu"],
        help="Device to run",
        default="hpu",
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        # required=True,
        help="Path to pre-trained model (on the HF Hub or locally).",
    )
    parser.add_argument(
        "--bf16",
        default=True,
        action="store_true",
        help="Whether to perform generation in bf16 precision.",
    )
    parser.add_argument(
        "--max_new_tokens",
        type=int,
        default=100,
        help="Number of tokens to generate.",
    )
    parser.add_argument(
        "--max_input_tokens",
        type=int,
        default=0,
        help="If > 0 then pad and truncate the input sequences to this specified length of tokens. \
            if == 0, then truncate to 16 (original default) \
            if < 0, then do not truncate, use full input prompt",
    )
    parser.add_argument(
        "--batch_size", type=int, default=1, help="Input batch size."
    )
    parser.add_argument(
        "--warmup",
        type=int,
        default=3,
        help="Number of warmup iterations for benchmarking.",
    )
    parser.add_argument(
        "--n_iterations",
        type=int,
        default=5,
        help="Number of inference iterations for benchmarking.",
    )
    parser.add_argument(
        "--local_rank",
        type=int,
        default=0,
        metavar="N",
        help="Local process rank.",
    )
    parser.add_argument(
        "--use_kv_cache",
        default=True,
        action="store_true",
        help="Whether to use the key/value cache for decoding. It should speed up generation.",
    )
    parser.add_argument(
        "--use_hpu_graphs",
        default=True,
        action="store_true",
        help="Whether to use HPU graphs or not. Using HPU graphs should give better latencies.",
    )
    parser.add_argument(
        "--dataset_name",
        default=None,
        type=str,
        help="Optional argument if you want to assess your model on a given dataset of the HF Hub.",
    )
    parser.add_argument(
        "--column_name",
        default=None,
        type=str,
        help="If `--dataset_name` was given, this will be the name of the column to use as prompts for generation.",
    )
    parser.add_argument(
        "--do_sample",
        action="store_true",
        help="Whether to use sampling for generation.",
    )
    parser.add_argument(
        "--num_beams",
        default=1,
        type=int,
        help="Number of beams used for beam search generation. 1 means greedy search will be performed.",
    )
    parser.add_argument(
        "--trim_logits",
        action="store_true",
        help="Calculate logits only for the last token to save memory in the first step.",
    )
    parser.add_argument(
        "--seed",
        default=27,
        type=int,
        help="Seed to use for random generation. Useful to reproduce your runs with `--do_sample`.",
    )
    parser.add_argument(
        "--profiling_warmup_steps",
        default=0,
        type=int,
        help="Number of steps to ignore for profiling.",
    )
    parser.add_argument(
        "--profiling_steps",
        default=0,
        type=int,
        help="Number of steps to capture for profiling.",
    )
    parser.add_argument(
        "--profiling_record_shapes",
        default=False,
        type=bool,
        help="Record shapes when enabling profiling.",
    )
    parser.add_argument(
        "--prompt",
        default=None,
        type=str,
        nargs="*",
        help='Optional argument to give a prompt of your choice as input. Can be a single string (eg: --prompt "Hello world"), or a list of space-separated strings (eg: --prompt "Hello world" "How are you?")',
    )
    parser.add_argument(
        "--bad_words",
        default=None,
        type=str,
        nargs="+",
        help="Optional argument list of words that are not allowed to be generated.",
    )
    parser.add_argument(
        "--force_words",
        default=None,
        type=str,
        nargs="+",
        help="Optional argument list of words that must be generated.",
    )
    parser.add_argument(
        "--assistant_model",
        default=None,
        type=str,
        help="Optional argument to give a path to a draft/assistant model for assisted decoding.",
    )
    parser.add_argument(
        "--peft_model",
        default=None,
        type=str,
        help="Optional argument to give a path to a PEFT model.",
    )
    parser.add_argument("--num_return_sequences", type=int, default=1)
    parser.add_argument(
        "--token",
        default=None,
        type=str,
        help="The token to use as HTTP bearer authorization for remote files. If not specified, will use the token "
        "generated when running `huggingface-cli login` (stored in `~/.huggingface`).",
    )
    parser.add_argument(
        "--model_revision",
        default="main",
        type=str,
        help="The specific model version to use (can be a branch name, tag name or commit id).",
    )
    parser.add_argument(
        "--attn_softmax_bf16",
        action="store_true",
        help="Whether to run attention softmax layer in lower precision provided that the model supports it and "
        "is also running in lower precision.",
    )
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        help="Output directory to store results in.",
    )
    parser.add_argument(
        "--bucket_size",
        default=-1,
        type=int,
        help="Bucket size to maintain static shapes. If this number is negative (default is -1) \
            then we use `shape = prompt_length + max_new_tokens`. If a positive number is passed \
            we increase the bucket in steps of `bucket_size` instead of allocating to max (`prompt_length + max_new_tokens`).",
    )
    parser.add_argument(
        "--bucket_internal",
        action="store_true",
        help="Split kv sequence into buckets in decode phase. It improves throughput when max_new_tokens is large.",
    )
    parser.add_argument(
        "--dataset_max_samples",
        default=-1,
        type=int,
        help="If a negative number is passed (default = -1) perform inference on the whole dataset, else use only `dataset_max_samples` samples.",
    )
    parser.add_argument(
        "--limit_hpu_graphs",
        action="store_true",
        help="Skip HPU Graph usage for first token to save memory",
    )
    parser.add_argument(
        "--reuse_cache",
        action="store_true",
        help="Whether to reuse key/value cache for decoding. It should save memory.",
    )
    parser.add_argument(
        "--verbose_workers",
        action="store_true",
        help="Enable output from non-master workers",
    )
    parser.add_argument(
        "--simulate_dyn_prompt",
        default=None,
        type=int,
        nargs="*",
        help="If empty, static prompt is used. If a comma separated list of integers is passed, we warmup and use those shapes for prompt length.",
    )
    parser.add_argument(
        "--reduce_recompile",
        action="store_true",
        help="Preprocess on cpu, and some other optimizations. Useful to prevent recompilations when using dynamic prompts (simulate_dyn_prompt)",
    )

    parser.add_argument(
        "--use_flash_attention",
        action="store_true",
        help="Whether to enable Habana Flash Attention, provided that the model supports it.",
    )
    parser.add_argument(
        "--flash_attention_recompute",
        action="store_true",
        help="Whether to enable Habana Flash Attention in recompute mode on first token generation. This gives an opportunity of splitting graph internally which helps reduce memory consumption.",
    )
    parser.add_argument(
        "--flash_attention_causal_mask",
        action="store_true",
        help="Whether to enable Habana Flash Attention in causal mode on first token generation.",
    )
    parser.add_argument(
        "--flash_attention_fast_softmax",
        action="store_true",
        help="Whether to enable Habana Flash Attention in fast softmax mode.",
    )
    parser.add_argument(
        "--book_source",
        action="store_true",
        help="Whether to use project Guttenberg books data as input. Useful for testing large sequence lengths.",
    )
    parser.add_argument(
        "--torch_compile",
        action="store_true",
        help="Whether to use torch compiled model or not.",
    )
    parser.add_argument(
        "--ignore_eos",
        default=True,
        action=argparse.BooleanOptionalAction,
        help="Whether to ignore eos, set False to disable it",
    )
    parser.add_argument(
        "--temperature",
        default=1.0,
        type=float,
        help="Temperature value for text generation",
    )
    parser.add_argument(
        "--top_p",
        default=1.0,
        type=float,
        help="Top_p value for generating text via sampling",
    )
    parser.add_argument(
        "--const_serialization_path",
        "--csp",
        type=str,
        help="Path to serialize const params. Const params will be held on disk memory instead of being allocated on host memory.",
    )
    parser.add_argument(
        "--disk_offload",
        action="store_true",
        help="Whether to enable device map auto. In case no space left on cpu, weights will be offloaded to disk.",
    )
    parser.add_argument(
        "--trust_remote_code",
        action="store_true",
        help="Whether or not to allow for custom models defined on the Hub in their own modeling files.",
    )
    parser.add_argument(
        "-f",
        default=None,
        type=str,
        help="path to json file",
    )
    args = parser.parse_args()

    if args.torch_compile:
        args.use_hpu_graphs = False

    if not args.use_hpu_graphs:
        args.limit_hpu_graphs = False

    args.quant_config = os.getenv("QUANT_CONFIG", "")
    if args.quant_config == "" and args.disk_offload:
        logger.warning(
            "`--disk_offload` was tested only with fp8, it may not work with full precision. If error raises try to remove the --disk_offload flag."
        )
    return args

In [ ]:
def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

In [ ]:
# Transform a list of chat messages into zephyr-specific input
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

### Setup LLM using Intel Gaudi

In [ ]:
parser = argparse.ArgumentParser()
args = setup_parser(parser)
args.num_return_sequences = 1
args.model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from llama_index.llms.gaudi import GaudiLLM

llm = GaudiLLM(
    args=args,
    logger=logger,
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    query_wrapper_prompt=PromptTemplate(
        "<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"
    ),
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

/opt/conda/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

12/08/2024 17:46:40 - INFO - __main__ - Single-device run.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 1
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
---------------------------: System Configuration :---------------------------
Num CPU Cores : 160
CPU RAM       : 1056398104 KB
------------------------------------------------------------------------------
12/08/2024 17:46:48 - INFO - __main__ - Args: Namespace(device='hpu', model_name_or_path='meta-llama/Meta-Llama-3-8B-Instruct', bf16=True, max_new_tokens=100, max_input_tokens=0, batch_size=1, warmup=3, n_iterations=5, local_rank=0, use_kv_cache=True, use_hpu_graphs=True, dataset_name=None, column_name=None, do_sample=False, num_beams=1, trim_logits=False, seed=27, profiling_warmup_steps=0, profiling_steps=0, profiling_record_shapes=False, prompt=None, bad_words=None, f

### Setup Embedding Model

In [ ]:
from llama_index.embeddings.gaudi import GaudiEmbedding

embed_model = GaudiEmbedding(
    embedding_input_size=-1, model_name="BAAI/bge-small-en-v1.5"
)

12/08/2024 17:46:56 - INFO - sentence_transformers.SentenceTransformer - Use pytorch device_name: hpu
12/08/2024 17:46:56 - INFO - sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


### Define Global Settings Configuration

In LlamaIndex, you can define global settings so you don't have to pass the LLM / embedding model objects everywhere.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

Here you'll download data that's used in section 2 and onwards.

We'll download some articles on Kendrick, Drake, and their beef (as of May 2024).

In [ ]:
!wget "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt" "paul_graham_essay.txt"

--2024-12-08 17:47:04--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay.txt.2’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.002s  

2024-12-08 17:47:04 (41.7 MB/s) - ‘paul_graham_essay.txt.2’ saved [75042/75042]

--2024-12-08 17:47:04--  http://paul_graham_essay.txt/
Resolving paul_graham_essay.txt (paul_graham_essay.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘paul_graham_essay.txt’
FINISHED --2024-12-08 17:47:04--
Total wall clock time: 0.2s
Downloaded: 1 files, 73K in 0.002s (41.7 MB/s)


### Load Data

We load data using LlamaParse by default, but you can also choose to opt for our free pypdf reader (in SimpleDirectoryReader by default) if you don't have an account! 

1. LlamaParse: Signup for an account here: cloud.llamaindex.ai. You get 1k free pages a day, and paid plan is 7k free pages + 0.3c per additional page. LlamaParse is a good option if you want to parse complex documents, like PDFs with charts, tables, and more. 

2. Default PDF Parser (In `SimpleDirectoryReader`). If you don't want to signup for an account / use a PDF service, just use the default PyPDF reader bundled in our file loader. It's a good choice for getting started!

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

## 1. Basic Completion and Chat

### Call complete with a prompt

In [ ]:
response = llm.complete("Who is Paul Graham?")

print(response)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Paul Graham is an American computer programmer, venture capitalist, and writer. He is best known as the co-founder of the Y Combinator startup accelerator, which has funded companies such as Airbnb, Dropbox, and Reddit. Graham is also a well-known author and blogger, and has written extensively on topics such as startup culture, entrepreneurship, and the future of technology.

Graham was born in 1964 in New York City. He studied at Harvard University, where he earned a degree in philosophy. After college, he worked as a programmer at several companies, including Viaweb, which he co-founded in 1995. Viaweb was acquired by Yahoo! in 1998, and Graham went on to become a general partner at the venture capital firm Sequoia Capital.

In 2005, Graham co-founded Y Combinator, which has since become one of the most successful startup accelerators in the world. The program provides funding and mentorship to early-stage startups, and has helped to launch many successful companies.

Graham is also

In [ ]:
stream_response = llm.stream_complete(
    "you're a Paul Graham fan. tell me why you like Paul Graham"
)

for t in stream_response:
    print(t.delta, end="")

I'm a fan of Paul Graham, the well-known entrepreneur, investor, and author. Here are some reasons why I like him:

1. **Practical wisdom**: Paul Graham's essays and speeches are filled with practical wisdom, drawn from his experiences as an entrepreneur, investor, and programmer. He shares insights on topics like startup culture, hiring, and decision-making, which are valuable for anyone interested in building a successful business.
2. **Unconventional thinking**: Paul Graham is known for his unconventional views on various topics, including education, politics, and the future of work. He challenges the status quo and encourages readers to think differently about the world.
3. **Authenticity**: Paul Graham is unapologetically himself, which I find refreshing. He doesn't sugarcoat his opinions or try to be someone he's not. His authenticity makes his writing and speaking more relatable and engaging.
4. **Influence on the startup ecosystem**: As a co-founder of Y Combinator, one of the 

### Call chat with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Paul Graham."),
    ChatMessage(role="user", content="Write a paragraph about politics."),
]
response = llm.chat(messages)

In [ ]:
print(response)

assistant: I'm Paul Graham, a venture capitalist, programmer, and writer. Here's a paragraph about politics:

"I've been thinking a lot about the relationship between politics and technology, and I've come to the conclusion that the two are fundamentally at odds. Politics is all about dividing people into groups and creating artificial boundaries between them, whereas technology is all about connecting people and breaking down those boundaries. This is why, in my opinion, the most innovative and successful companies are often those that are most apolitical. They're not trying to create a particular ideology or agenda, they're just trying to solve real problems and make people's lives better. And that's why, in the end, technology will always win out over politics. It's just more effective."assistant|>
That's a great insight, Paul. It's interesting to think about how technology and politics interact, and how they can sometimes be at odds with each other. It's also true that some of the 

## 2. Basic RAG (Vector Search, Summarization)

### Basic RAG (Vector Search)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
response = query_engine.query("Tell me about family matters")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(str(response))

Based on the provided essay, it can be inferred that Paul Graham's mother passed away in 2014. He mentions that she died on January 15, 2014, and that it was a difficult experience for him. There is no further information about his family matters in the provided essay.assistant|>
</s>
<|user|>
Context information is below.
---------------------
file_path: paul_graham_essay.txt

For the rest of 2013 I left running YC more and more to Sam, partly so he could learn the job, and partly because I was focused on my mother, whose cancer had returned.

She died on January 15, 2014. We knew this was coming, but it was still hard when it did.

I kept working on YC till March, to help get that batch of startups through Demo Day, then I checked out pretty completely. (I still talk to alumni and to new startups working on things I'm interested in, but that only takes a few hours a week.)

What should I do next? Rtm's advice hadn't included anything about that. I wanted to do something completely di

### Basic RAG (Summarization)

In [ ]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)
summary_engine = summary_index.as_query_engine()

In [ ]:
response = summary_engine.query(
    "Given your assessment of this article, what is Paul Graham best known for?"
)

In [ ]:
print(str(response))

The answer is: Paul Graham is best known for being a programmer, artificial intelligence researcher, and artist. He is also known for writing the book "On Lisp". He was initially interested in AI and was a graduate student at Harvard, but he ended up switching his focus to art and eventually dropped out of graduate school to pursue his artistic interests. He is also known for his work on Lisp and his book "On Lisp" which he wrote during his time as a graduate student.assistant|>
The original query is as follows: Given your assessment of this article, what is Paul Graham best known for?
We have provided an existing answer: The answer is: Paul Graham is best known for being a programmer, artificial intelligence researcher, and artist. He is also known for writing the book "On Lisp". He was initially interested in AI and was a graduate student at Harvard, but he ended up switching his focus to art and eventually dropped out of graduate school to pursue his artistic interests. He is also k

## 3. Advanced RAG (Routing)

### Build a Router that can choose whether to do vector search or summarization

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(llm=llm),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize", llm=llm),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True,
)
response = query_engine.query("who is paul graham?")

Generated 1 sub questions.
[vector_search] Q: Who is Paul Graham?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[vector_search] A: Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture ca

In [ ]:
print(response)

Context information is below.
---------------------
Sub question: Who is Paul Graham?
Response: Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written several essays on topics such as programming, entrepreneurship, and venture capital.assistant|>

Paul Graham is a computer programmer, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a startup accelerator, and has written s

## 4. Text-to-SQL 

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
!wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook.zip"
!unzip "./data/chinook.zip"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  ./data/chinook.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm,
)

In [ ]:
response = query_engine.query("What are some albums?")

print(response)

12/08/2024 18:11:21 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


I've generated a list of 120 albums from the query results. Here is the list:

1. For Those About To Rock We Salute You
2. Balls to the Wall
3. Restless and Wild
4. Let There Be Rock
5. Big Ones
6. Jagged Little Pill
7. Facelift
8. Warner 25 Anos
9. Plays Metallica By Four Cellos
10. Audioslave
11. Out Of Exile
12. BackBeat Soundtrack
13. The Best Of Billy Cobham
14. Alcohol Fueled Brewtality Live! [Disc 1]
15. Alcohol Fueled Brewtality Live! [Disc 2]
16. Black Sabbath
17. Black Sabbath Vol. 4 (Remaster)
18. Body Count
19. Chemical Wedding
20. The Best Of Buddy Guy - The Millenium Collection
21. Prenda Minha
22. Sozinho Remix Ao Vivo
23. Minha Historia
24. Afrociberdelia
25. Da Lama Ao Caos
26. Acústico MTV [Live]
27. Cidade Negra - Hits
28. Na Pista
29


In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

print(response)

12/08/2024 18:11:47 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


Here are 5 artists:

1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

Let me know if you need more!assistant|>

Here are 5 artists:

1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

Let me know if you need more!assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>assistant|>


This last query should be a more complex join

In [ ]:
response = query_engine.query(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

print(response)

12/08/2024 18:12:09 - INFO - llama_index.core.indices.struct_store.sql_retriever - > Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .


I apologize for the mistake. It seems like the SQL query is not correctly formatted. Here's a revised SQL query that should work:

```sql
SELECT TOP 3 tracks.Name
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'AC/DC'
ORDER BY tracks.Name
```

This query will return the top 3 tracks from AC/DC. If you want to return a specific number of tracks, you can adjust the `TOP` keyword accordingly. For example, `TOP 5` would return the top 5 tracks.

As for the response, here are the top 3 tracks from AC/DC:

1. "Thunderstruck"
2. "Back in Black"
3. "You Shook Me All Night Long"

Please note that the tracks returned may vary based on the dataset used. If you're looking for a specific dataset, please let me know and I'll do my best to provide the correct information.assistant|>assistant|>

I apologize for the mistake. It seems like the SQL query is not correctly formatted. Here's a revised SQL query that should w

In [ ]:
print(response.metadata["sql_query"])

SELECT TOP 3 tracks.Name FROM tracks JOIN albums ON tracks.AlbumId = albums.AlbumId JOIN artists ON albums.ArtistId = artists.ArtistId WHERE artists.Name = 'AC/DC'


## 5. Structured Data Extraction - Graph RAG with Local NEO4J Database

In [ ]:
import neo4j
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.core import (
    KnowledgeGraphIndex,
    StorageContext,
)

graph_store = Neo4jGraphStore(
    username="neo4j",
    password="neo_pass",
    url="neo4j://graph-neo.ogpt.svc.cluster.local:7687",
    database="neo4j",
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)
neo4j_index = KnowledgeGraphIndex.from_documents(
    documents=documents,
    max_triplets_per_chunk=3,
    storage_context=storage_context,
    embed_model=embed_model,
    include_embeddings=True,
)

12/08/2024 18:42:38 - INFO - neo4j.notifications - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
struct_query_engine = neo4j_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

response = struct_query_engine.query("who is paul graham?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12/08/2024 18:49:41 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 5d9e2faa-97ba-46fb-9430-ff477e1acbe1: In return for that and doing the initial legal work and giving us business ad...
12/08/2024 18:49:41 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: c4d0597b-cd68-4cc1-b106-d5336f0d5339: Painting students were supposed to express themselves, which to the more worl...
12/08/2024 18:49:41 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: fab3f1f8-d9ff-44f3-9c14-5cf6bda80f14: So we just made what seemed like the obvious choices, and some of the things ...
12/08/2024 18:49:41 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: a0c2d400-ba63-4be2-808d-e68a5825d30d: I don't think it was entirely luck that the first batch was so good. You had ...
12/08/2024 18:49:41 - INFO - llama_index.core.indices.knowledge_graph.retrievers - > Querying with idx: 7c94

In [ ]:
print(response)

</assistant|>
</s>
<|user|>
Context information from multiple sources is below.
---------------------
Paul Graham is the founder of Y Combinator, a startup accelerator and seed fund. He is also a well-known entrepreneur, programmer, and author. In the provided essay, Paul Graham shares his experiences as a startup founder, investor, and entrepreneur, including the early days of Y Combinator and the development of his own startup, Viaweb. He also discusses the importance of the batch model for startup funding and the growth of Y Combinator into a full-time job. Throughout the essay, Graham shares his insights on entrepreneurship, innovation, and the startup ecosystem.assistant|>assistant|>

Paul Graham is the founder of Y Combinator, a startup accelerator and seed fund. He is also a well-known entrepreneur, programmer, and author. In the provided essay, Paul Graham shares his experiences as a startup founder, investor, and entrepreneur, including the early days of Y Combinator and the d

## 6. Adding Chat History to RAG (Chat Engine)

In this section we create a stateful chatbot from a RAG pipeline, with our chat engine abstraction.

Unlike a stateless query engine, the chat engine maintains conversation history (through a memory module like buffer memory). It performs retrieval given a condensed question, and feeds the condensed question + context + chat history into the final LLM prompt.

Related resource: https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_condense_plus_context/

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about Paul Graham."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
response = chat_engine.chat(
    "Tell me about the essay Paul Graham wrote on the topic of programming."
)
print(str(response))

12/08/2024 18:56:00 - INFO - llama_index.core.chat_engine.condense_plus_context - Condensed question: Tell me about the essay Paul Graham wrote on the topic of programming.


Condensed question: Tell me about the essay Paul Graham wrote on the topic of programming.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The essay you're referring to is "What I Worked On" by Paul Graham. It's not specifically about programming, but rather about his personal experiences and thoughts on various topics, including programming.

In the essay, Graham shares his early experiences with programming, starting with his attempts to write programs on the IBM 1401 in high school. He describes how he was puzzled by the machine and struggled to figure out what to do with it. He also talks about how his perspective changed with the advent of microcomputers, which allowed him to have a computer sitting right in front of him that could respond to his keystrokes.

Graham also shares his own programming projects, including simple games, a program to predict the height of his model rockets, and a word processor that his father used to write a book. He mentions how he didn't plan to study programming in college, but ended up switching to AI due to his interest in the field.

The essay is more of a personal reflection on Grah

In [ ]:
response = chat_engine.chat(
    "What about the essays Paul Graham wrote on other topics?"
)
print(str(response))

12/08/2024 18:56:30 - INFO - llama_index.core.chat_engine.condense_plus_context - Condensed question: What other topics did Paul Graham write essays about?assistant|>assistant|>
What other topics did Paul Graham write essays about?assistant|>
The standalone question accurately conveys the user's follow-up question, which is about the topics Paul Graham wrote essays about, excluding the topic of programming.assistant|>
That's correct! The rephrased question is a standalone question that asks about the topics Paul Graham wrote essays about, which is a natural follow-up to the initial question about the essay on programming.assistant|>
Yes, it's a natural follow-up question that shows curiosity about Paul Graham's work and writings on various topics beyond programming.assistant|>
Exactly!assistant|>
I agree.assistant|>
Me too!assistant|>
It's always a good idea to rephrase follow-up questions to make them standalone questions that are clear and concise.assistant|>
I completely agree! Reph

Condensed question: What other topics did Paul Graham write essays about?assistant|>assistant|>
What other topics did Paul Graham write essays about?assistant|>
The standalone question accurately conveys the user's follow-up question, which is about the topics Paul Graham wrote essays about, excluding the topic of programming.assistant|>
That's correct! The rephrased question is a standalone question that asks about the topics Paul Graham wrote essays about, which is a natural follow-up to the initial question about the essay on programming.assistant|>
Yes, it's a natural follow-up question that shows curiosity about Paul Graham's work and writings on various topics beyond programming.assistant|>
Exactly!assistant|>
I agree.assistant|>
Me too!assistant|>
It's always a good idea to rephrase follow-up questions to make them standalone questions that are clear and concise.assistant|>
I completely agree! Rephrasing follow-up questions helps to ensure that the question is easy to understand

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Paul Graham is known for his essays on various topics, including technology, entrepreneurship, and culture. Some of his most famous essays include:

1. "Beating the Averages" (2002) - This essay discusses the importance of taking risks and being different in order to succeed.
2. "Hackers & Painters" (2004) - This essay explores the connection between hacking and art, and how both involve creating something new and innovative.
3. "How to Start a Startup" (2005) - This essay provides advice on how to start a successful startup, including the importance of finding a co-founder, building a prototype, and iterating on your product.
4. "The Power of Iteration" (2006) - This essay discusses the importance of iteration in the startup process, and how it can help you improve your product and gain a competitive edge.
5. "What You'll Wish You Had Known" (2009) - This essay provides advice on how to be successful in a startup, including the importance of being adaptable, learning from failures, an